#### Download and install TensorFlow2 package. Import TensorFlow into your program

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

#### Load and prepare the MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

#### Use `tf.data` to batch and shuffle the dataset

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

#### Build the `tf.keras` model using the Keras model subclassing API

In [11]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
# Create an instance of the model 
model = MyModel()

#### Choose an optimizer and loss function for training

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#### Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result

In [13]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#### Use `tf.GradientTape` to train the model

In [20]:
@tf.function

def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

#### Test the model

In [21]:
@tf.function   # what is the role of @ in front of tf? (@tf)   

def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [22]:
EPOCHS = 5

for epoch in range(EPOCHS):
    #Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    template = 'Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch: 1, Loss: 0.13417157530784607, Accuracy: 96.00166320800781, Test Loss: 0.06213216856122017, Test Accuracy: 97.94999694824219
Epoch: 2, Loss: 0.04262320324778557, Accuracy: 98.68999481201172, Test Loss: 0.05101914703845978, Test Accuracy: 98.29999542236328
Epoch: 3, Loss: 0.022091424092650414, Accuracy: 99.27166748046875, Test Loss: 0.0612160786986351, Test Accuracy: 98.08999633789062
Epoch: 4, Loss: 0.014704354107379913, Accuracy: 99.48666381835938, Test Loss: 0.05743560567498207, Test Accuracy: 98.4000015258789
Epoch: 5, Loss: 0.009001877158880234, Accuracy: 99.69999694824219, Test Loss: 0.06537005305290222, Test Accuracy: 98.47000122070312


end -

##### reference 
https://www.tensorflow.org/tutorials/quickstart/advanced